In [193]:
import pandas as pd
pd.set_option('display.max_columns',40)
import os
os.chdir("C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_100%_profit")

In [194]:
# Path to your data folder in Google Drive
data_folder = "C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_100%_profit"
# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['updated_distnce_ideal_target(WIPRO)_dataset4.csv']

In [303]:
s = pd.read_csv('updated_distnce_ideal_target(WIPRO)_dataset4.csv')

In [39]:
# s = pd.read_csv('Processed_5%_RF_5%_results (2).csv')
# s1 = pd.read_csv('Processed_10%_RF_10%_results (2).csv')
# s2 = pd.read_csv('Processed_20%_RF_20%_results (2).csv')
# s3 = pd.read_csv('Processed_50%_RF_50%_results (2).csv')
# s4 = pd.read_csv('Processed_80%_RF_80%_results (2).csv')
# s5 = pd.read_csv('Processed_100%_RF_100%_results (3).csv')

# print(s.duplicated().sum())
# print(s1.duplicated().sum())
# print(s2.duplicated().sum())
# print(s3.duplicated().sum())
# print(s4.duplicated().sum())
# print(s5.duplicated().sum())


In [196]:
s.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Time Stamp', 'Open', 'High', 'Low',
       'Close', 'SellersLevel4', 'SellersLevel3', 'SellersLevel2',
       'SellersLevel1', 'SellersLevel0', 'BuyersLevel0', 'BuyersLevel1',
       'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4', 'BuyersInControlPrice',
       'SellersInControlPrice', 'Entry', 'Defence', 'Stop', 'Target',
       'Actual_100%_dis', 'Predicted_100%_dis', 'Predicted_25%_dis',
       'Predicted_50%_dis', 'Predicted_75%_dis', 'Actual_Ideal',
       'Predicted_Ideal_25%', 'Predicted_Ideal_50%', 'Predicted_Ideal_75%',
       'Predicted_Ideal_100%', 'Profit', 'Actual_ideal_profit'],
      dtype='object')

In [197]:
s.shape

(2337, 35)

In [6]:
s.duplicated().sum()

0

In [7]:
s.drop_duplicates(inplace=True)

In [8]:
s.duplicated().sum()

0

In [198]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add the prefix 'PIT_'
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 250
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                    

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low

                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)

        # Additional Calculations
        df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
        df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

    return df

def process_files(input_dir, output_dir):
    # List all CSV files in the input directory
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    for file in files:
        # Extract the percentage from the filename
        percent = file.split('_')[1].replace('%', '')

        # Load the DataFrame
        df = pd.read_csv(os.path.join(input_dir, file))

        # Determine the Target_Predicted_X_dis column to use
        target_col = f'Predicted_Ideal_100%'

        # Apply trading logic
        df = apply_trading_logic(df, target_col, percent)

        # Drop unnecessary columns before saving
        columns_to_drop = [
            f'PIT_Target_Buyer_{percent}',
            f'PIT_Defence_Target_Buyer_{percent}',
            f'PIT_Target_Seller_{percent}',
            f'PIT_Defence_Target_Seller_{percent}'
        ]
        df.drop(columns=columns_to_drop, inplace=True)

        # Save the processed DataFrame to the output directory
        output_file = f'Processed_{percent}_{file}'
        df.to_csv(os.path.join(output_dir, output_file), index=False)

# Example usage
output_dir = "C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_100%_prediction"
process_files(data_folder, output_dir)


  0%|          | 0/2337 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\767695003.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '521.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\767695003.py:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.76' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
  3%|▎         | 69/2337 [00:00<00:03, 684.77it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\767695003.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '288.349

For 25%

In [213]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add the prefix 'PIT_'
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 250
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                    

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low

                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)

        # Additional Calculations
        df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
        df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

    return df

def process_files(input_dir, output_dir):
    # List all CSV files in the input directory
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    for file in files:
        # Extract the percentage from the filename
        percent = file.split('_')[1].replace('%', '')

        # Load the DataFrame
        df = pd.read_csv(os.path.join(input_dir, file))

        # Determine the Target_Predicted_X_dis column to use
        target_col = f'Predicted_Ideal_25%'

        # Apply trading logic
        df = apply_trading_logic(df, target_col, percent)

        # Drop unnecessary columns before saving
        columns_to_drop = [
            f'PIT_Target_Buyer_{percent}',
            f'PIT_Defence_Target_Buyer_{percent}',
            f'PIT_Target_Seller_{percent}',
            f'PIT_Defence_Target_Seller_{percent}'
        ]
        df.drop(columns=columns_to_drop, inplace=True)

        # Save the processed DataFrame to the output directory
        output_file = f'Processed_{percent}_{file}'
        df.to_csv(os.path.join(output_dir, output_file), index=False)

# Example usage
output_dir = "C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_25%_prediction"
process_files(data_folder, output_dir)


  0%|          | 0/2337 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2626831644.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '521.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2626831644.py:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.23' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
  3%|▎         | 64/2337 [00:00<00:03, 634.61it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2626831644.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '281.7

For 50%

In [228]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add the prefix 'PIT_'
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 250
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                    

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low

                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)

        # Additional Calculations
        df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
        df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

    return df

def process_files(input_dir, output_dir):
    # List all CSV files in the input directory
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    for file in files:
        # Extract the percentage from the filename
        percent = file.split('_')[1].replace('%', '')

        # Load the DataFrame
        df = pd.read_csv(os.path.join(input_dir, file))

        # Determine the Target_Predicted_X_dis column to use
        target_col = f'Predicted_Ideal_50%'

        # Apply trading logic
        df = apply_trading_logic(df, target_col, percent)

        # Drop unnecessary columns before saving
        columns_to_drop = [
            f'PIT_Target_Buyer_{percent}',
            f'PIT_Defence_Target_Buyer_{percent}',
            f'PIT_Target_Seller_{percent}',
            f'PIT_Defence_Target_Seller_{percent}'
        ]
        df.drop(columns=columns_to_drop, inplace=True)

        # Save the processed DataFrame to the output directory
        output_file = f'Processed_{percent}_{file}'
        df.to_csv(os.path.join(output_dir, output_file), index=False)

# Example usage
output_dir = "C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_50%_prediction"
process_files(data_folder, output_dir)


  0%|          | 0/2337 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\1752869353.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '521.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\1752869353.py:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.41' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
  3%|▎         | 63/2337 [00:00<00:03, 613.26it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\1752869353.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '288.3

For 75%


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add the prefix 'PIT_'
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 250
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                    

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low

                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)

        # Additional Calculations
        df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
        df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

    return df

def process_files(input_dir, output_dir):
    # List all CSV files in the input directory
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    for file in files:
        # Extract the percentage from the filename
        percent = file.split('_')[1].replace('%', '')

        # Load the DataFrame
        df = pd.read_csv(os.path.join(input_dir, file))

        # Determine the Target_Predicted_X_dis column to use
        target_col = f'Predicted_Ideal_75%'

        # Apply trading logic
        df = apply_trading_logic(df, target_col, percent)

        # Drop unnecessary columns before saving
        columns_to_drop = [
            f'PIT_Target_Buyer_{percent}',
            f'PIT_Defence_Target_Buyer_{percent}',
            f'PIT_Target_Seller_{percent}',
            f'PIT_Defence_Target_Seller_{percent}'
        ]
        df.drop(columns=columns_to_drop, inplace=True)

        # Save the processed DataFrame to the output directory
        output_file = f'Processed_{percent}_{file}'
        df.to_csv(os.path.join(output_dir, output_file), index=False)

# Example usage
output_dir = "C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_50%_prediction"
process_files(data_folder, output_dir)


For 75%(count of else part)


In [281]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add the prefix 'PIT_'
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0

    # Counter to track unmatched defence cases
    counter_buyer = 0
    counter_seller=0
    
    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    # Updating PIT columns for Buyers
                    df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                    df.loc[i, f'PIT_Aggresive_Target_{percent}'] = max_high
                else:
                    # Increment counter if no defence match is found
                    counter_buyer += 1

            

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 250
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
                                    

            # Sellers Logic
           
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low
                else:
                    counter_seller+=1
               


                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] =abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)
    print(f"Total number of times matched_low_index for seller was empty: {counter_seller}")
    print(f"Total number of times matched_low_index for buyer was empty: {counter_buyer}")

    #     # Additional Calculations
    #     df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
    #     df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

    # return df

def process_files(input_dir):
    # List all CSV files in the input directory
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    for file in files:
        # Extract the percentage from the filename
        percent = file.split('_')[1].replace('%', '')

        # Load the DataFrame
        df = pd.read_csv(os.path.join(input_dir, file))

        # Determine the Target_Predicted_X_dis column to use
        target_col = f'Predicted_Ideal_75%'

        # Apply trading logic
        df = apply_trading_logic(df, target_col, percent)

        # Drop unnecessary columns before saving
        # columns_to_drop = [
        #     f'PIT_Target_Buyer_{percent}',
        #     f'PIT_Defence_Target_Buyer_{percent}',
        #     f'PIT_Target_Seller_{percent}',
        #     f'PIT_Defence_Target_Seller_{percent}'
        # ]
        # df.drop(columns=columns_to_drop, inplace=True)


#         # Save the processed DataFrame to the output directory
#         output_file = f'Processed_{percent}_{file}'
#         df.to_csv(os.path.join(output_dir, output_file), index=False)

# # Example usage
# output_dir = "C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_75%_prediction"
# process_files(data_folder, output_dir)


In [282]:
a ="C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_100%_profit"
c =  process_files(a)

  0%|          | 0/2337 [00:00<?, ?it/s]

C:\Users\dell\AppData\Local\Temp\ipykernel_15364\4157153115.py:82: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '521.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\4157153115.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '284.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\4157153115.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '249.25' has dtype incompatible with int64, please explicitly cast to a compatible d

Total number of times matched_low_index for seller was empty: 131
Total number of times matched_low_index for buyer was empty: 0


In [ ]:
# import os
# import pandas as pd
# from tqdm import tqdm

# def apply_trading_logic(df, target_col, percent):
#     # Initialize new columns with 0 and add timestamp columns
#     df[f'PIT_Profit_{percent}'] = 0
#     df[f'PIT_D_Profit_{percent}'] = 0
#     df[f'PIT_Loss_{percent}'] = 0
#     df[f'PIT_Target_Buyer_{percent}'] = 0
#     df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
#     df[f'PIT_Target_Seller_{percent}'] = 0
#     df[f'PIT_Defence_Target_Seller_{percent}'] = 0
#     df[f'PIT_Defence_Target_{percent}'] = 0
#     df[f'PIT_Aggresive_Target_{percent}'] = 0

#     # Add timestamp columns
#     df[f'PIT_Profit_Timestamp_{percent}'] = ''
#     df[f'PIT_D_Profit_Timestamp_{percent}'] = ''
#     df[f'PIT_Loss_Timestamp_{percent}'] = ''

#     # Iterate over each row with tqdm for progress tracking
#     for i in tqdm(range(len(df))):
#         row = df.iloc[i]

#         # Entry Value Check
#         if row['Entry'] != 0:
#             entry = row['Entry']
#             defence = row['Defence']

#             # Buyers Logic
#             if row['BuyersInControlPrice'] != 0:
#                 # Defence Check
#                 low_future_rows = df.iloc[i+1:][['Low', 'Time Stamp']]
#                 matched_low_index = low_future_rows[low_future_rows['Low'] == defence].index
#                 if not matched_low_index.empty:
#                     matched_low_index = matched_low_index[0]
#                     max_high = df.loc[i:matched_low_index, 'High'].max()
#                     max_high_timestamp = df.loc[df['High'] == max_high, 'Time Stamp'].values[0]
#                     max_close = df.loc[i:matched_low_index, 'Close'].max()

#                     df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
#                     df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

#                     # Target_Predicted_X_dis vs. Max High Value Comparison
#                     if row[target_col] <= max_high:
#                         df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry), 2)
#                         df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = max_high_timestamp
#                     else:
#                         stop = row['Stop']
#                         if stop != 0:
#                             df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = ((entry + defence) / 2) + 250
#                             matched_stop_index = low_future_rows[low_future_rows['Low'] == stop].index
#                             if not matched_stop_index.empty:
#                                 matched_stop_index = matched_stop_index[0]
#                                 max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
#                                 max_high_stop_timestamp = df.loc[df['High'] == max_high_stop, 'Time Stamp'].values[0]

#                                 # Defence Target vs. Max High Value Comparison
#                                 if df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] <= max_high_stop:
#                                     df.loc[i, f'PIT_D_Profit_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
#                                     df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = max_high_stop_timestamp
#                                 else:
#                                     df.loc[i, f'PIT_Loss_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] - entry)
#                                     df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = max_high_stop_timestamp

#             # Sellers Logic
#             if row['SellersInControlPrice'] != 0:
#                 # Defence Check
#                 high_future_rows = df.iloc[i+1:][['High', 'Time Stamp']]
#                 matched_high_index = high_future_rows[high_future_rows['High'] == defence].index
#                 if not matched_high_index.empty:
#                     matched_high_index = matched_high_index[0]
#                     min_low = df.loc[i:matched_high_index, 'Low'].min()
#                     min_low_timestamp = df.loc[df['Low'] == min_low, 'Time Stamp'].values[0]
#                     min_close = df.loc[i:matched_high_index, 'Close'].min()

#                     df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
#                     df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low

#                     # Target_Predicted_X_dis vs. Min Low Value Comparison
#                     if row[target_col] >= min_low:
#                         df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]), 2)
#                         df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = min_low_timestamp
#                     else:
#                         stop = row['Stop']
#                         if stop != 0:
#                             df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = ((entry + defence) / 2) - 250
#                             matched_stop_index = high_future_rows[high_future_rows['High'] == stop].index
#                             if not matched_stop_index.empty:
#                                 matched_stop_index = matched_stop_index[0]
#                                 min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
#                                 min_low_stop_timestamp = df.loc[df['Low'] == min_low_stop, 'Time Stamp'].values[0]

#                                 # Defence Target vs. Min Low Value Comparison
#                                 if df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] >= min_low_stop:
#                                     df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'PIT_Defence_Target_Seller_{percent}'])
#                                     df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = min_low_stop_timestamp
#                                 else:
#                                     df.loc[i, f'PIT_Loss_{percent}'] = abs(df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] - entry)
#                                     df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = min_low_stop_timestamp

#         # Additional Calculations
#         df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
#         df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

#     return df


In [322]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 0 and add timestamp columns
    df[f'PIT_Profit_{percent}'] = 0
    df[f'PIT_D_Profit_{percent}'] = 0
    df[f'PIT_Loss_{percent}'] = 0
    df[f'PIT_Target_Buyer_{percent}'] = 0
    df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
    df[f'PIT_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_Seller_{percent}'] = 0
    df[f'PIT_Defence_Target_{percent}'] = 0
    df[f'PIT_Aggresive_Target_{percent}'] = 0
    df[f'PIT_Ideal_Target_{percent}'] = 0

    # Add timestamp columns
    df[f'PIT_Profit_Timestamp_{percent}'] = ''
    df[f'PIT_D_Profit_Timestamp_{percent}'] = ''
    df[f'PIT_Loss_Timestamp_{percent}'] = ''

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            print("entry",entry)
            defence = row['Defence']
            stop = row['Stop']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                low_future_rows = df.iloc[i + 1:][['Low', 'Time Stamp']]
                matched_low_index = low_future_rows[low_future_rows['Low'] <= defence].index
                print("---------------------------------------")
                print("matched_low_index",matched_low_index)
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    print("max_high",max_high)
                    max_high_timestamp = df.loc[df['High'] == max_high, 'Time Stamp'].values[0]
                    print("max_high_timestamp",max_high_timestamp)
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    # Update Target values
                    df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

                    # Check for predicted target comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry), 2)
                        df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = max_high_timestamp
                    else:
                        if stop != 0:
                            defence_target = ((entry + defence) / 2) + 250
                            df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = defence_target
                            matched_stop_index = low_future_rows[low_future_rows['Low'] <= stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_high_stop_timestamp = df.loc[df['High'] == max_high_stop, 'Time Stamp'].values[0]

                                # Defence Target vs. Max High Value Comparison
                                if defence_target <= max_high_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(defence_target - entry)
                                    df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = max_high_stop_timestamp
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] = abs(defence_target - entry)
                                    df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = max_high_stop_timestamp

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                high_future_rows = df.iloc[i + 1:][['High', 'Time Stamp']]
                matched_high_index = high_future_rows[high_future_rows['High'] >= defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_low_timestamp = df.loc[df['Low'] == min_low, 'Time Stamp'].values[0]
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    # Update Target values
                    df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
                    df.loc[i, f'PIT_Aggresive_T_{percent}'] = min_low

                    # Check for predicted target comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PIT_Profit_{percent}'] = round((entry - row[target_col]), 2)
                        df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = min_low_timestamp
                    else:
                        if stop != 0:
                            defence_target = ((entry + defence) / 2) - 250
                            df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = defence_target
                            matched_stop_index = high_future_rows[high_future_rows['High'] >= stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_low_stop_timestamp = df.loc[df['Low'] == min_low_stop, 'Time Stamp'].values[0]

                                # Defence Target vs. Min Low Value Comparison
                                if defence_target >= min_low_stop:
                                    df.loc[i, f'PIT_D_Profit_{percent}'] = abs(entry - defence_target)
                                    df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = min_low_stop_timestamp
                                else:
                                    df.loc[i, f'PIT_Loss_{percent}'] = abs(defence_target - entry)
                                    df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = min_low_stop_timestamp

    # Calculate Ideal and Defence Targets
    df.loc[i, f'PIT_Ideal_Target_{percent}'] = df.loc[i, f'PIT_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Target_Seller_{percent}']
    df.loc[i, f'PIT_Defence_Target_{percent}'] = df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] + df.loc[i, f'PIT_Defence_Target_Seller_{percent}']

    return df


In [347]:
df = pd.read_csv('updated_distnce_ideal_target(WIPRO)_dataset4.csv')

In [348]:
df.sample(20)

,Unnamed: 0.1,Unnamed: 0,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlPrice,SellersInControlPrice,Entry,Defence,Stop,Target,Actual_100%_dis,Predicted_100%_dis,Predicted_25%_dis,Predicted_50%_dis,Predicted_75%_dis,Actual_Ideal,Predicted_Ideal_25%,Predicted_Ideal_50%,Predicted_Ideal_75%,Predicted_Ideal_100%,Profit,Actual_ideal_profit
465,465,465,2024-10-10 10:30:00,536.85,537.60,536.70,537.40,532.40,532.25,531.95,525.20,524.80,531.70,532.50,532.95,533.10,534.25,537.40,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
459,459,459,2024-10-10 10:00:00,534.90,535.00,534.15,534.35,532.40,532.25,531.95,525.20,524.80,531.50,531.70,532.50,532.95,533.10,0.00,534.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1051,1051,1051,2024-10-22 09:20:00,551.95,551.95,548.10,550.00,550.50,548.30,543.50,533.20,533.10,551.85,551.90,553.15,553.30,555.30,0.00,550.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,1140,1140,2024-10-23 10:30:00,551.00,553.50,550.55,550.75,547.50,547.00,546.80,546.40,544.20,544.80,546.00,546.65,547.25,548.10,0.00,550.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1117,1117,1117,2024-10-22 14:50:00,546.85,547.00,546.20,546.20,548.40,548.30,548.00,547.50,546.80,547.25,549.55,551.85,551.90,553.15,0.00,546.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1894,1894,1894,2024-11-07 09:50:00,564.50,564.50,562.50,562.95,559.45,559.30,543.95,542.65,542.20,558.80,561.00,561.20,562.20,563.45,0.00,562.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334,334,334,2024-10-08 12:05:00,523.65,523.65,522.30,522.45,525.25,524.80,524.45,523.65,523.00,520.30,521.70,522.65,524.70,528.70,0.00,522.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1321,1321,1321,2024-10-25 13:05:00,543.80,544.40,543.15,544.00,547.65,547.25,547.15,547.00,546.75,542.20,542.35,542.80,543.20,543.55,544.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1263,1263,1263,2024-10-24 14:30:00,548.25,548.25,547.80,547.85,547.25,547.15,547.00,546.75,546.55,545.70,546.40,546.65,546.75,546.80,0.00,547.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1840,1840,1840,2024-11-06 11:35:00,559.05,560.25,558.95,559.95,559.45,543.95,542.65,542.20,541.85,542.85,547.00,547.15,557.00,558.80,559.95,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [346]:
import os
import pandas as pd
from tqdm import tqdm

# Initialize columns
percent = 100
target_col = 'Predicted_Ideal_100%'
df[f'PIT_Profit_{percent}'] = 0
df[f'PIT_D_Profit_{percent}'] = 0
df[f'PIT_Loss_{percent}'] = 0
df[f'PIT_Target_Buyer_{percent}'] = 0
df[f'PIT_Defence_Target_Buyer_{percent}'] = 0
df[f'PIT_Target_Seller_{percent}'] = 0
df[f'PIT_Defence_Target_Seller_{percent}'] = 0
df[f'PIT_Defence_Target_{percent}'] = 0
df[f'PIT_Aggresive_Target_{percent}'] = 0
df[f'PIT_Ideal_Target_{percent}'] = 0

# Timestamp columns
df[f'PIT_Profit_Timestamp_{percent}'] = ''
df[f'PIT_D_Profit_Timestamp_{percent}'] = ''
df[f'PIT_Loss_Timestamp_{percent}'] = ''

# Iterate over each row with tqdm for progress tracking
for i in tqdm(range(len(df))):
    row = df.iloc[i]

    # Check if 'Entry' is not zero
    if row['Entry'] != 0:
        entry = row['Entry']
        defence = row['Defence']
        stop = row['Stop']
        
        # print("=--------------------------------------------")

        # Buyers Logic
        if row['BuyersInControlPrice'] != 0:
            # print("-------------------------------")

            # Check for future rows where 'Low' is less than or equal to 'defence'
            low_future_rows = df.iloc[i + 1:][['Low', 'Time Stamp']]
            matched_low_index = low_future_rows[low_future_rows['Low'] <= defence].index
            print("matched_low_index", matched_low_index)

            if not matched_low_index.empty:
                matched_low_index = matched_low_index[0]

                # Calculate max 'High' from current index to matched index
                max_high = df.loc[i:matched_low_index, 'High'].max()
                print("max_high", max_high)
                
                # Timestamp for the max high
                max_high_timestamp = df.loc[df['High'] == max_high, 'Time Stamp'].values[0]
                print("max_high_timestamp", max_high_timestamp)
                
                # Calculate max 'Close' from current index to matched index
                max_close = df.loc[i:matched_low_index, 'Close'].max()

                # Update Target values
                df.loc[i, f'PIT_Target_Buyer_{percent}'] = max_close
                df.loc[i, f'PIT_Aggresive_T_{percent}'] = max_high

                # Check for predicted target comparison
                if row[target_col] <= max_high:
                    df.loc[i, f'PIT_Profit_{percent}'] = round((row[target_col] - entry), 2)
                    df.loc[i, f'PIT_Profit_Timestamp_{percent}'] = max_high_timestamp
                else:
                    if stop != 0:
                        # Calculate defence target
                        defence_target = ((entry + defence) / 2) + 250
                        df.loc[i, f'PIT_Defence_Target_Buyer_{percent}'] = defence_target
                        
                        # Check for future rows where 'Low' is less than or equal to 'stop'
                        matched_stop_index = low_future_rows[low_future_rows['Low'] <= stop].index

                        if not matched_stop_index.empty:
                            matched_stop_index = matched_stop_index[0]

                            # Calculate max 'High' from current index to stop matched index
                            max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                            max_high_stop_timestamp = df.loc[df['High'] == max_high_stop, 'Time Stamp'].values[0]

                            # Defence Target vs. Max High Value Comparison
                            if defence_target <= max_high_stop:
                                df.loc[i, f'PIT_D_Profit_{percent}'] = abs(defence_target - entry)
                                df.loc[i, f'PIT_D_Profit_Timestamp_{percent}'] = max_high_stop_timestamp
                            else:
                                df.loc[i, f'PIT_Loss_{percent}'] = abs(defence_target - entry)
                                df.loc[i, f'PIT_Loss_Timestamp_{percent}'] = max_high_stop_timestamp


100%|██████████| 2337/2337 [00:00<00:00, 6773.35it/s]


In [305]:
s.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Time Stamp', 'Open', 'High', 'Low',
       'Close', 'SellersLevel4', 'SellersLevel3', 'SellersLevel2',
       'SellersLevel1', 'SellersLevel0', 'BuyersLevel0', 'BuyersLevel1',
       'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4', 'BuyersInControlPrice',
       'SellersInControlPrice', 'Entry', 'Defence', 'Stop', 'Target',
       'Actual_100%_dis', 'Predicted_100%_dis', 'Predicted_25%_dis',
       'Predicted_50%_dis', 'Predicted_75%_dis', 'Actual_Ideal',
       'Predicted_Ideal_25%', 'Predicted_Ideal_50%', 'Predicted_Ideal_75%',
       'Predicted_Ideal_100%', 'Profit', 'Actual_ideal_profit'],
      dtype='object')

In [323]:
e=apply_trading_logic(s, "Predicted_Ideal_100%", 100)

  0%|          | 0/2337 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2612902928.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '546.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Target_Seller_{percent}'] = min_close
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2612902928.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '296.67499999999995' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'PIT_Defence_Target_Seller_{percent}'] = defence_target
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2612902928.py:107: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '249.72500000000002' has dtype incompatible w

entry 546.4
entry 546.6
entry 546.65
entry 546.6
entry 546.05
entry 537.4
entry 530.85
entry 530.8
entry 531.35
entry 532.55
entry 532.5
entry 531.75
entry 531.6
entry 530.65
entry 530.55
entry 529.7
entry 530.25
entry 529.8
entry 528.8
entry 531.3
entry 533.35
entry 534.8
entry 539.05
entry 538.15
entry 539.1
entry 537.95


 13%|█▎        | 315/2337 [00:00<00:02, 739.01it/s] 

entry 536.5
entry 537.45
entry 536.55
entry 536.7
entry 535.55
entry 535.35
entry 534.9
entry 534.7
entry 535.3
entry 535.5
entry 535.4
entry 534.25
entry 532.95
entry 532.2
entry 531.75
entry 532.2
entry 531.4
entry 530.1
entry 529.45
entry 529.25
entry 530.3
entry 529.05
entry 530.0
entry 530.55
entry 524.85
entry 524.85


 24%|██▎       | 551/2337 [00:00<00:01, 993.20it/s]

...

entry 573.6
entry 573.4
entry 572.7
entry 569.7
entry 570.0
entry 570.35
entry 569.8
entry 569.65
entry 569.6
entry 568.2
entry 567.8
entry 567.95
entry 567.5
entry 568.0


 97%|█████████▋| 2256/2337 [00:03<00:00, 400.06it/s]

entry 568.25
entry 567.75
entry 569.1
entry 568.4
entry 568.2
entry 569.2
entry 569.3
entry 568.95
entry 568.25
entry 567.65
entry 567.9
entry 568.7
entry 568.35
entry 567.75
entry 570.0
entry 569.5
entry 569.25
entry 568.6
entry 569.3
entry 568.75
entry 567.85
entry 566.55


100%|██████████| 2337/2337 [00:03<00:00, 742.96it/s]

entry 567.0
entry 566.35
entry 566.45
entry 566.0


In [307]:
e.drop(columns=['Unnamed: 0.1','Unnamed: 0'],inplace=True,axis=1)

In [308]:
e.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Predicted_25%_dis', 'Predicted_50%_dis',
       'Predicted_75%_dis', 'Actual_Ideal', 'Predicted_Ideal_25%',
       'Predicted_Ideal_50%', 'Predicted_Ideal_75%', 'Predicted_Ideal_100%',
       'Profit', 'Actual_ideal_profit', 'PIT_Profit_100', 'PIT_D_Profit_100',
       'PIT_Loss_100', 'PIT_Target_Buyer_100', 'PIT_Defence_Target_Buyer_100',
       'PIT_Target_Seller_100', 'PIT_Defence_Target_Seller_100',
       'PIT_Defence_Target_100', 'PIT_Aggresive_Target_100',
       'PIT_Ideal_Target_100', 'PIT_Profit_Timestamp_100',
       'PIT_D_Profit_Timestamp_100', 'PIT_Loss_Timestamp_100',
       'PIT_Aggresive_T_100'

In [309]:
e.to_csv("C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\rough_files_csv,rough2.csv",index=False)

In [245]:
data = pd.read_csv("C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_75%_prediction\\Processed_distnce_updated_distnce_ideal_target(WIPRO)_dataset4.csv")

In [246]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Time Stamp', 'Open', 'High', 'Low',
       'Close', 'SellersLevel4', 'SellersLevel3', 'SellersLevel2',
       'SellersLevel1', 'SellersLevel0', 'BuyersLevel0', 'BuyersLevel1',
       'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4', 'BuyersInControlPrice',
       'SellersInControlPrice', 'Entry', 'Defence', 'Stop', 'Target',
       'Actual_100%_dis', 'Predicted_100%_dis', 'Predicted_25%_dis',
       'Predicted_50%_dis', 'Predicted_75%_dis', 'Actual_Ideal',
       'Predicted_Ideal_25%', 'Predicted_Ideal_50%', 'Predicted_Ideal_75%',
       'Predicted_Ideal_100%', 'Profit', 'Actual_ideal_profit',
       'PIT_Profit_distnce', 'PIT_D_Profit_distnce', 'PIT_Loss_distnce',
       'PIT_Defence_Target_distnce', 'PIT_Aggresive_Target_distnce',
       'PIT_Ideal_Target_distnce', 'PIT_Aggresive_T_distnce'],
      dtype='object')

In [247]:
import pandas as pd
from tqdm import tqdm
import os

def apply_trading_logic(df):
    # Initialize the new columns with 0
    df['T_Profit'] = 0
    df['D_Profit'] = 0
    df['Loss'] = 0
    df['Ideal_Target_Buyer'] = 0
    df['Defence_Target_Buyer'] = 0
    df['Ideal_Target_Seller'] = 0
    df['Defence_Target_Seller'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, 'Ideal_Target_Buyer'] = max_close
                    # df.loc[i, 'Ideal_Target_Agressive'] = max_high

                    # Target vs. Max High Value Comparison
                    if row['Target'] <= max_high:
                        df.loc[i, 'T_Profit'] = abs(row['Target'] - entry)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Buyer'] = ((entry + defence) / 2) + 250
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, 'Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, 'D_Profit'] = abs(df.loc[i, 'Defence_Target_Buyer'] - entry)
                                else:
                                    df.loc[i, 'Loss'] = abs(df.loc[i, 'Defence_Target_Buyer'] - entry)

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, 'Ideal_Target_Seller'] = min_close
                    # df.loc[i, 'Ideal_Target_Agressive'] = min_low

                    # Target vs. Min Low Value Comparison
                    if row['Target'] >= min_low:
                        df.loc[i, 'T_Profit'] = abs(entry - row['Target'])
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Seller'] = ((entry + defence) / 2) - 250
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, 'Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, 'D_Profit'] = abs(entry - df.loc[i, 'Defence_Target_Seller'])
                                else:
                                    df.loc[i, 'Loss'] = abs(df.loc[i, 'Defence_Target_Seller'] - entry)

        # Additional Calculations
        # df.loc[i, 'Ideal_Target'] = df.loc[i, 'Ideal_Target_Buyer'] + df.loc[i, 'Ideal_Target_Seller']
        # df.loc[i, 'Defence_Target'] = df.loc[i, 'Defence_Target_Buyer'] + df.loc[i, 'Defence_Target_Seller']

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Ideal_Target_Buyer", "Ideal_Target_Seller", "Defence_Target_Buyer", "Defence_Target_Seller"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    return df

In [248]:
data = apply_trading_logic(data)

  0%|          | 0/2337 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2776314787.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '521.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Ideal_Target_Seller'] = min_close
C:\Users\dell\AppData\Local\Temp\ipykernel_15364\2776314787.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0499999999999545' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'T_Profit'] = abs(entry - row['Target'])
100%|██████████| 2337/2337 [00:00<00:00, 3447.67it/s]


In [249]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Time Stamp', 'Open', 'High', 'Low',
       'Close', 'SellersLevel4', 'SellersLevel3', 'SellersLevel2',
       'SellersLevel1', 'SellersLevel0', 'BuyersLevel0', 'BuyersLevel1',
       'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4', 'BuyersInControlPrice',
       'SellersInControlPrice', 'Entry', 'Defence', 'Stop', 'Target',
       'Actual_100%_dis', 'Predicted_100%_dis', 'Predicted_25%_dis',
       'Predicted_50%_dis', 'Predicted_75%_dis', 'Actual_Ideal',
       'Predicted_Ideal_25%', 'Predicted_Ideal_50%', 'Predicted_Ideal_75%',
       'Predicted_Ideal_100%', 'Profit', 'Actual_ideal_profit',
       'PIT_Profit_distnce', 'PIT_D_Profit_distnce', 'PIT_Loss_distnce',
       'PIT_Defence_Target_distnce', 'PIT_Aggresive_Target_distnce',
       'PIT_Ideal_Target_distnce', 'PIT_Aggresive_T_distnce', 'T_Profit',
       'D_Profit', 'Loss'],
      dtype='object')

In [250]:
data.drop(columns=['T_Profit', 'D_Profit'],axis=1,inplace=True)

In [251]:
data.drop(columns=['Unnamed: 0'],inplace=True,axis=1)

In [252]:
data.drop(columns=['Unnamed: 0.1'],inplace=True,axis=1)

In [253]:
data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Predicted_25%_dis', 'Predicted_50%_dis',
       'Predicted_75%_dis', 'Actual_Ideal', 'Predicted_Ideal_25%',
       'Predicted_Ideal_50%', 'Predicted_Ideal_75%', 'Predicted_Ideal_100%',
       'Profit', 'Actual_ideal_profit', 'PIT_Profit_distnce',
       'PIT_D_Profit_distnce', 'PIT_Loss_distnce',
       'PIT_Defence_Target_distnce', 'PIT_Aggresive_Target_distnce',
       'PIT_Ideal_Target_distnce', 'PIT_Aggresive_T_distnce', 'Loss'],
      dtype='object')

For 25%

In [224]:
data.drop(columns=['Predicted_50%_dis','Predicted_75%_dis','Predicted_Ideal_75%','Predicted_Ideal_50%','Predicted_Ideal_100%'],inplace=True,axis=1)

For 50%

In [239]:
data.drop(columns=['Predicted_25%_dis','Predicted_75%_dis','Predicted_Ideal_25%','Predicted_Ideal_75%','Predicted_Ideal_100%'],inplace=True,axis=1)

For 75%

In [254]:
data.drop(columns=['Predicted_25%_dis','Predicted_50%_dis','Predicted_Ideal_25%','Predicted_Ideal_50%','Predicted_Ideal_100%'],inplace=True,axis=1)

For 100%


In [209]:
data.drop(columns=['Predicted_25%_dis','Predicted_50%_dis','Predicted_75%_dis','Predicted_Ideal_25%','Predicted_Ideal_50%','Predicted_Ideal_75%'],inplace=True,axis=1)

In [255]:
data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Predicted_75%_dis', 'Actual_Ideal',
       'Predicted_Ideal_75%', 'Profit', 'Actual_ideal_profit',
       'PIT_Profit_distnce', 'PIT_D_Profit_distnce', 'PIT_Loss_distnce',
       'PIT_Defence_Target_distnce', 'PIT_Aggresive_Target_distnce',
       'PIT_Ideal_Target_distnce', 'PIT_Aggresive_T_distnce', 'Loss'],
      dtype='object')

In [256]:
data.to_csv("C:\\Users\\dell\\Desktop\\Exp_Dist(WIPRO)\\Exp_dist_dataset4_75%_prediction\\Processed_distnce_updated_distnce_ideal_target_1_(WIPRO).csv",index=False)

**Aggressive**


In [1]:
import pandas as pd
pd.set_option('display.max_columns',40)
import os
os.chdir("C:\\Users\\dell\\Desktop\\Non_imputed\\Non_imputed_100%_profit")

In [2]:
# Path to your data folder in Google Drive
data_folder_1 = "C:\\Users\\dell\\Desktop\\Non_imputed\\Non_imputed_100%_profit"

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder_1)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['Processed_100%_newModeltest_Agressive_data_non_imputed (1).csv']

In [3]:
s = pd.read_csv("Processed_100%_newModeltest_Agressive_data_non_imputed (1).csv")
# s1 = pd.read_csv("Processed_10%_Updated_RF_Agressive_10%_results (1).csv")
# s2 = pd.read_csv("Processed_20%_Updated_RF_Agressive_20%_results (1).csv")
# s3 = pd.read_csv("Processed_50%_Updated_RF_Agressive_50%_results (1).csv")
# s4 = pd.read_csv("Processed_80%_Updated_RF_Agressive_80%_results (1).csv")
# s5 = pd.read_csv("Processed_100%_Updated_RF_Agressive_100%_results (1).csv")

In [4]:
print(s.columns)
# print(s1.columns)
# print(s2.columns)
# print(s3.columns)
# print(s4.columns)
# print(s5.columns)

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'Actual_100%_dis',
       'Predicted_100%_dis', 'Actual_Ideal', 'Predicted_Ideal', 'Profit'],
      dtype='object')


In [5]:
# s.columns

In [6]:
s.shape

(85172, 26)

In [7]:
s.duplicated().sum()

22

In [9]:
s.drop_duplicates(inplace=True)

In [10]:
s.duplicated().sum()

0

In [11]:
import os
import pandas as pd
from tqdm import tqdm

def apply_trading_logic(df, target_col, percent):
    # Initialize new columns with 'Aggressive' in the column names
    df[f'PAT_Profit_{percent}'] = 0
    df[f'PAT_D_Profit_{percent}'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Loss'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Target_Buyer'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Target_Seller'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Defence_Target'] = 0
    df[f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Buyer'] = max_close
                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = max_high

                    # Target_Predicted_X_dis vs. Max High Value Comparison
                    if row[target_col] <= max_high:
                        df.loc[i, f'PAT_Profit_{percent}'] = round(abs(row[target_col] - entry),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, f'PAT_D_Profit_{percent}'] = abs(df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] - entry)
                                else:
                                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Loss'] = abs(df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] - entry)

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Seller'] = min_close
                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = min_low

                    # Target_Predicted_X_dis vs. Min Low Value Comparison
                    if row[target_col] >= min_low:
                        df.loc[i, f'PAT_Profit_{percent}'] = round(abs(entry - row[target_col]),2)
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] = ((entry + defence) / 2) - 0.25
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, f'PAT_D_Profit_{percent}'] = abs(entry - df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'])
                                else:
                                    df.loc[i, f'Target_Predicted_Aggressive_{percent}_Loss'] = abs(df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'] - entry)

        # Additional Calculations
        df.loc[i, f'Target_Predicted_Aggressive_{percent}_Ideal_T'] = df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Buyer'] + df.loc[i, f'Target_Predicted_Aggressive_{percent}_Target_Seller']
        df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target'] = df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer'] + df.loc[i, f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller']

    return df

def process_files(input_dir, output_dir):
    # List all CSV files in the input directory
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    for file in files:
        # Extract the percentage from the filename
        percent = file.split('_')[1].replace('%', '')

        # Load the DataFrame
        df = pd.read_csv(os.path.join(input_dir, file))

        # Determine the Target_Predicted_X_dis column to use
        target_col = f'Predicted_Agressive_{percent}_dis'
        # Target_Predicted_Agressive_100_dis

        # Apply trading logic
        df = apply_trading_logic(df, target_col, percent)

        # Drop unnecessary columns before saving
        columns_to_drop = [
            f'Target_Predicted_Aggressive_{percent}_Target_Buyer',
            f'Target_Predicted_Aggressive_{percent}_Defence_Target_Buyer',
            f'Target_Predicted_Aggressive_{percent}_Target_Seller',
            f'Target_Predicted_Aggressive_{percent}_Defence_Target_Seller'
        ]
        df.drop(columns=columns_to_drop, inplace=True)

        # Save the processed DataFrame to the output directory
        output_file = f'Processed_{percent}_{file}'
        df.to_csv(os.path.join(output_dir, output_file), index=False)

# Example usage

output_dir = "C:\\Users\\dell\\Desktop\\Non_imputed\\Non_imputed_100%_prediction"
process_files(data_folder_1, output_dir)


  0%|          | 0/85172 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_16472\1267156648.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4638.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, f'Target_Predicted_Aggressive_{percent}_Aggresive_T'] = max_high
  0%|          | 4/85172 [00:00<04:14, 334.62it/s]


KeyError: 'Predicted_Agressive_100_dis'